In [1]:
from datetime import datetime
from geopy.distance import geodesic

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from matplotlib.colors import ListedColormap

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import  plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score


# **1. Data Preprocessing**

In [2]:
taxi_trip = pd.read_csv('../data/raw/nyc-yellow-taxi-trip-2016.csv',
                parse_dates=['pickup_datetime'])

display(taxi_trip.head(3))

print("Shape:", taxi_trip.shape)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124


Shape: (1458644, 11)


Let's check the dataset information, and see if there is any missing value or duplicated rows:

In [3]:
print(taxi_trip.info())
print("\n Duplicated rows:", taxi_trip.duplicated().sum())
print("\n Missing values:", taxi_trip.isnull().sum().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1458644 non-null  object        
 1   vendor_id           1458644 non-null  int64         
 2   pickup_datetime     1458644 non-null  datetime64[ns]
 3   dropoff_datetime    1458644 non-null  object        
 4   passenger_count     1458644 non-null  int64         
 5   pickup_longitude    1458644 non-null  float64       
 6   pickup_latitude     1458644 non-null  float64       
 7   dropoff_longitude   1458644 non-null  float64       
 8   dropoff_latitude    1458644 non-null  float64       
 9   store_and_fwd_flag  1458644 non-null  object        
 10  trip_duration       1458644 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 122.4+ MB
None

 Duplicated rows: 0

 Missing values: 0


There are no missing values or duplicated rows. The only feature that we need to change is `store_and_fwd_flag`, which is a categorical feature. We will change it to a numerical feature mapping `Y` to 1 and `N` to 0: 

In [4]:
# Map store_and_fwd_flag to 0 and 1 
taxi_trip['store_and_fwd_flag'] = taxi_trip['store_and_fwd_flag'].map({'N': 0, 'Y': 1})

## 1.2 Feature Engineering

Before steeping into the exploratory data analyses (EDA), is necessary to create new features that will help us to better understand the data. 

### 1.2.1 Datetime Features

Using the pickup time with format `YYYY-MM-DD H:M:S`, let’s extract features like the year, month, day, hours and minutes for more informative features and also the time of day, day of the week, day of the year to help with possible underlying patters in the data related to specific time intervals. So we will create a total of 8 new features:

In [5]:
# Days in a year: 1-365 (or 366)
taxi_trip['day_of_year'] = taxi_trip['pickup_datetime'].dt.dayofyear
# Day of week : 0-6, where 0 is Monday and 6 Sunday
taxi_trip['day_of_week'] = taxi_trip['pickup_datetime'].dt.dayofweek
# Minutes in a day : 0-1440
taxi_trip['min_of_day'] = (taxi_trip['pickup_datetime'].dt.hour * 60 + taxi_trip['pickup_datetime'].dt.minute)

# Date features
taxi_trip['year'] = taxi_trip['pickup_datetime'].dt.year
taxi_trip['month'] = taxi_trip['pickup_datetime'].dt.month
taxi_trip['day'] = taxi_trip['pickup_datetime'].dt.day
taxi_trip['hour'] = taxi_trip['pickup_datetime'].dt.hour
taxi_trip['minute'] = taxi_trip['pickup_datetime'].dt.minute

In [6]:
taxi_trip.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,day_of_year,day_of_week,min_of_day,year,month,day,hour,minute
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,455,74,0,1044,2016,3,14,17,24
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,663,164,6,43,2016,6,12,0,43
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,19,1,695,2016,1,19,11,35
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,429,97,2,1172,2016,4,6,19,32
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,435,86,5,810,2016,3,26,13,30


Another relevant information is about weekends and holidays. For this we will use the [holiday' dataset from kaggle](https://www.kaggle.com/datasets/pceccon/nyc2016holidays/data), which contains the new york city holidays for the year of 2016. We will use this dataset to create a new binary feature called `is_holiday`, and another for the weekend called `is_weekend`:

In [7]:
holidays = pd.read_csv('../data/raw/nyc-holidays-2016.csv', sep = ';')
display(holidays.head())

,Day,Date,Holiday
0,Friday,January 01,New Years Day
1,Monday,January 18,Martin Luther King Jr. Day
2,Friday,February 12,Lincoln's Birthday
3,Monday,February 15,Presidents' Day
4,Sunday,May 08,Mother's Day


In [8]:
# Convert to datetime the Date column
holidays['Date'] = pd.to_datetime(holidays['Date'] + ', 2016')

# Extract month and day as numerical values
holidays['month'] = holidays['Date'].dt.month
holidays['day'] = holidays['Date'].dt.day
# Add the is_holiday column 
holidays['is_holiday'] = True

# Drop the original columns
holidays.drop(['Date', 'Day', 'Holiday'], axis=1, inplace=True)

# Save the holidays dataset
holidays.to_csv('../data/processed/nyc-holidays-2016.csv',index=False)

display(holidays.head())

,month,day,is_holiday
0,1,1,True
1,1,18,True
2,2,12,True
3,2,15,True
4,5,8,True


In [9]:
# Merge the taxi data with the holidays data
taxi_trip = taxi_trip.merge(holidays, on=['month', 'day'], how='left')

# Replace NaN values in the is_holiday column with False
taxi_trip['is_holiday'].fillna(False, inplace=True)

# Add the is_weekend column (5:Saturday or 6:Sunday)
taxi_trip['is_weekend'] = taxi_trip['day_of_week'].isin([5, 6])

display(taxi_trip.head())
print('Number of trips in holiday:', taxi_trip[taxi_trip['is_holiday'] == True].shape[0])

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,day_of_year,day_of_week,min_of_day,year,month,day,hour,minute,is_holiday,is_weekend
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,74,0,1044,2016,3,14,17,24,False,False
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,164,6,43,2016,6,12,0,43,False,True
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,19,1,695,2016,1,19,11,35,False,False
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,97,2,1172,2016,4,6,19,32,False,False
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,86,5,810,2016,3,26,13,30,False,True


Number of trips in holiday: 51122


In [10]:
taxi_trip[taxi_trip['hour'] == 2]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,day_of_year,day_of_week,min_of_day,year,month,day,hour,minute,is_holiday,is_weekend
20,id2070428,1,2016-02-28 02:23:02,2016-02-28 02:31:08,1,-73.980370,40.742420,-73.962852,40.760635,0,...,59,6,143,2016,2,28,2,23,False,True
60,id1674872,1,2016-02-29 02:46:24,2016-02-29 02:55:10,1,-73.951851,40.714069,-73.967972,40.688721,0,...,60,0,166,2016,2,29,2,46,False,False
195,id3707411,1,2016-01-03 02:03:39,2016-01-03 02:17:02,1,-74.007729,40.740608,-73.983513,40.737209,0,...,3,6,123,2016,1,3,2,3,False,True
228,id2718231,1,2016-03-08 02:44:19,2016-03-08 03:04:35,1,-73.992500,40.740444,-73.840111,40.719517,0,...,68,1,164,2016,3,8,2,44,False,False
290,id3792892,1,2016-05-01 02:40:38,2016-05-01 03:02:30,1,-73.986664,40.721684,-73.972572,40.685879,0,...,122,6,160,2016,5,1,2,40,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458206,id3439636,2,2016-01-31 02:24:49,2016-01-31 02:44:58,4,-73.987930,40.728275,-74.004089,40.751850,0,...,31,6,144,2016,1,31,2,24,False,True
1458248,id3247855,1,2016-04-02 02:37:14,2016-04-02 02:39:36,1,-74.002396,40.734116,-74.003754,40.737980,0,...,93,5,157,2016,4,2,2,37,False,True
1458395,id2750534,1,2016-01-10 02:24:22,2016-01-10 02:57:46,2,-74.004150,40.752171,-73.990623,40.671833,0,...,10,6,144,2016,1,10,2,24,False,True
1458424,id0998702,2,2016-03-06 02:15:18,2016-03-06 02:24:16,1,-73.963203,40.671833,-73.960808,40.648785,0,...,66,6,135,2016,3,6,2,15,False,True


### 1.2.2 Traffic Features

With a briefly search in google about the traffic of New york city I found some useful information from a [Guide to Traffic in NYC](https://www.uber.com/blog/new-york-city/the-drivers-guide-to-traffic-in-a-new-york-minute/) and in [10 Tips to Beat NYC Traffic](https://intellishift.com/resources/blog/10-tips-to-beat-nyc-traffic/). The high traffic times to travel in and out of Manhattan are between 8-9 a.m. and 3-7 p.m. So we will create a new feature called `is_rush_hour` to indicate if the pickup time is in the rush hour or not in a day of the week. For simplicity, in the weekend and holidays we will consider the whole day as normal hour of traffic.

In [11]:
# Define rush hours
morning_rush = (taxi_trip['hour'] >= 8) & (taxi_trip['hour'] < 9)
evening_rush = (taxi_trip['hour'] >= 15) & (taxi_trip['hour'] < 20)

# Weekdays are 0 (Monday) to 4 (Friday)
is_weekday = taxi_trip['day_of_week'] < 5


taxi_trip['is_rush_hour'] = (morning_rush | evening_rush) & is_weekday

display(taxi_trip.head())

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,day_of_week,min_of_day,year,month,day,hour,minute,is_holiday,is_weekend,is_rush_hour
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,0,1044,2016,3,14,17,24,False,False,True
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,6,43,2016,6,12,0,43,False,True,False
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,1,695,2016,1,19,11,35,False,False,False
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,2,1172,2016,4,6,19,32,False,False,True
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,5,810,2016,3,26,13,30,False,True,False


### 1.2.3 Distance Features

In [12]:
def geodesic_distance(pickup_lat, pickup_long, dropoff_lat, dropoff_long):
    distances = [ geodesic( (lat1, lon1), (lat2, lon2) ).km for lat1, lon1, lat2, lon2 
                                                            in  zip(pickup_lat, pickup_long, 
                                                                    dropoff_lat, dropoff_long) ]
    return distances

taxi_trip['geodesic_distance'] = geodesic_distance( taxi_trip['pickup_latitude'].to_numpy(), 
                                                    taxi_trip['pickup_longitude'].to_numpy(), 
                                                    taxi_trip['dropoff_latitude'].to_numpy(), 
                                                    taxi_trip['dropoff_longitude'].to_numpy())

In [14]:
display(taxi_trip)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,min_of_day,year,month,day,hour,minute,is_holiday,is_weekend,is_rush_hour,geodesic_distance
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,1044,2016,3,14,17,24,False,False,True,1.502172
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,43,2016,6,12,0,43,False,True,False,1.808660
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,695,2016,1,19,11,35,False,False,False,6.379687
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,1172,2016,4,6,19,32,False,False,True,1.483632
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,810,2016,3,26,13,30,False,True,False,1.187038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,0,...,811,2016,4,8,13,31,False,False,False,1.227090
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,0,...,455,2016,1,10,7,35,False,True,False,6.046212
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,0,...,417,2016,4,22,6,57,False,False,False,7.821532
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,0,...,956,2016,1,5,15,56,False,False,True,1.092524


In [15]:
def bearing(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Calculate the change in coordinates
    dlon = lon2 - lon1

    # Calculate bearing
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    initial_bearing = np.arctan2(x, y)

    # Convert from radians to degrees and normalize
    initial_bearing = np.degrees(initial_bearing)
    
    # Normalize to 0-360
    bearing = (initial_bearing + 360) % 360

    return bearing

taxi_trip['bearing'] = bearing( taxi_trip['pickup_latitude'], taxi_trip['pickup_longitude'], 
                                taxi_trip['dropoff_latitude'], taxi_trip['dropoff_longitude'])

display(taxi_trip.head())

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,year,month,day,hour,minute,is_holiday,is_weekend,is_rush_hour,geodesic_distance,bearing
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,2016,3,14,17,24,False,False,True,1.502172,99.970196
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,2016,6,12,0,43,False,True,False,1.808660,242.846232
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,2016,1,19,11,35,False,False,False,6.379687,200.319835
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,2016,4,6,19,32,False,False,True,1.483632,187.262300
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,2016,3,26,13,30,False,True,False,1.187038,179.473585


### 1.2.4 Weather Features

[weather dataset from kaggle](https://www.kaggle.com/datasets/cabaki/knycmetars2016)

In [46]:
weather = pd.read_csv('../data/raw/nyc-weather-2016.csv', parse_dates=['Time'])
weather['Conditions'].unique()

array(['Overcast', 'Clear', 'Partly Cloudy', 'Mostly Cloudy',
       'Scattered Clouds', 'Unknown', 'Light Rain', 'Haze', 'Rain',
       'Heavy Rain', 'Light Snow', 'Snow', 'Heavy Snow',
       'Light Freezing Fog', 'Light Freezing Rain', 'Fog'], dtype=object)

In [28]:
weather.isnull().sum()  

Time             0
Temp.            0
Windchill     6492
Heat Index    7972
Humidity         0
Pressure       231
Dew Point        0
Visibility     237
Wind Dir         0
Wind Speed       0
Gust Speed       0
Precip           0
Events           0
Conditions       0
year             0
month            0
day              0
hour             0
dtype: int64

In [47]:
weather['year'] = weather['Time'].dt.year
weather['month'] = weather['Time'].dt.month
weather['day'] = weather['Time'].dt.day
weather['hour'] = weather['Time'].dt.hour

# Select only 2016
weather = weather[(weather['year'] == 2016)]

weather.drop([ 'Time', 'Windchill', 'Heat Index', 'Humidity', 'Pressure', 'Dew Point', 'Wind Dir',
                'Wind Speed', 'Gust Speed', 'Events'], axis=1, inplace=True)

display(weather.head())

,Temp.,Visibility,Precip,Conditions,year,month,day,hour
22,5.6,16.1,0.0,Overcast,2016,1,1,0
23,5.6,16.1,0.0,Overcast,2016,1,1,1
24,5.6,16.1,0.0,Overcast,2016,1,1,2
25,5.0,16.1,0.0,Overcast,2016,1,1,3
26,5.0,16.1,0.0,Overcast,2016,1,1,4


Number of rows per month: 11.825439783491204
